##  Lenet5 Models -JackieN 
This File Produces A number of Lenet5 Models and Predictions based on varying degrees of cleaned Train data.

Based on https://medium.com/@mgazar/lenet-5-in-9-lines-of-code-using-keras-ac99294c8086 and 

https://deepai.org/publication/towards-good-practices-on-building-effective-cnn-baseline-model-for-person-re-identification#:~:text=The%20last%20key%20practice%20is%20to%20train%20CNN,based%20on%20the%20adaptive%20estimates%20of%20lower-order%20moments.

The best score produced from the model using the clean data with all outliers removed is: 3.23581  

Placing at position 72 on the leaderboard

![](https://i.imgur.com/JwXgz3C.jpg)
 

In [ ]:
import os
import imp
import pickle
import pandas as pd
from utils import predict_models, load_models, transform_data


In [ ]:
### this is gold right here.
imp.reload(load_models)
file_path = "C:/Data/Augmented/"
trainer = load_models.LoadTrainModels("C:/data/Jackie_Lenet5_Aug", file_path)

trainer.print_paths()

### For every version of a clean Train file in a given path, create and save a model

In [ ]:
imp.reload(load_models)

file_path = "C:/Data/Augmented/"
#files = os.listdir("../CleanTrain")
files = os.listdir(file_path)

#For every version of a cleaned Train file in CleanTrain directory, create and save a model
for filename in files: 
    print("Opening file: ", filename)
    clean_file = "".join((file_path,filename))
    train_data = pickle.load( open( clean_file, "rb" ) )
    train_data = train_data.drop(['level_0', 'check_sum', 'index'], axis=1,errors='ignore')
    print("Train Shape:", train_data.shape)

    filename = str(filename).replace('.p', '').strip()
    print("Begin model and train:")
    model_name = "".join((filename,"_Lenet5"))
    print("Model name:", model_name)
    model, history = trainer.train_lenet5(model_name, train_data,verbose = True)
    print("End model and train")    
    print()

### For every model file in a given path, predict using the model and save the predictions in CSV file

In [ ]:

imp.reload(predict_models)
imp.reload(transform_data)

id_lookup = pickle.load( open( "Data/id_lookup.p", "rb" ) )
test = pickle.load( open( "Data/test.p", "rb" ) )

#Using local paths as this is way faster...
file_path = "C:/Data/Jackie_Lenet5_Aug"
pred_path = "C:/data/Predictions_Aug/"

predictor = predict_models.PredictModels(file_path,pred_path , id_lookup)

predictor.print_paths()

files = os.listdir(file_path)
#For every model in file_path, predict using the model and save the predictions in CSV file
for filename in files:
    if ".h5" in filename:
        base_name = filename[:-3]
        model_json = ''.join((base_name,".json"))
        print("Working with: ", base_name)
        print("Begin Predict")
        Y= predictor.predict_lenet5(base_name, filename, model_json, test)
        print("End model and train")    
        print()


## Combining Predictions using full columns vs separate predictions plus new predictions

In [ ]:
file_path = "data/predictions/"
files = os.listdir(file_path)

#All of the output rows for the submission file <= 17592 are keypoints for images with > 8 keypoints
#All rows following this only require 6 or 8 keypoints
max_row_full = 17592

#For all the different prediction files, combine predictions from the 30/full model with the 8/separate model
for file in files:
    if "Lenet5" in file:
        if "flip" in file:
            print(file)
            predictions_separate= pd.read_csv("data/predictions/"+file)
            file = file.replace("_flip","")
            predictions_full = pd.read_csv("data/Predictions_l5/"+file)
            new_pred = predictions_separate[predictions_separate['RowId']>max_row_full].copy()
            new_pred = new_pred.append(predictions_full[predictions_full['RowId']<=max_row_full].copy())
            new_pred = new_pred.sort_values(by=['RowId'])
            new_pred.to_csv("combined_"+"_flip"+file,index=False)